In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
from scipy.interpolate import CubicSpline
plt.rc('text', usetex=True)
plt.rc('font', family='Serif')

import matplotlib as mpl 
mpl.rcParams['figure.figsize'] = [9, 9]
mpl.rcParams['font.size'] = 27

mpl.rcParams['savefig.dpi'] = 150 
mpl.rcParams['xtick.minor.visible'] = True
mpl.rcParams['ytick.minor.visible'] = True
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'

mpl.rcParams['xtick.top'] = True
mpl.rcParams['ytick.right'] = True

mpl.rcParams['xtick.major.size'] = 10 
mpl.rcParams['xtick.minor.size'] = 5 

mpl.rcParams['ytick.major.size'] = 10 
mpl.rcParams['ytick.minor.size'] = 5 

mpl.rcParams['xtick.labelsize'] = 23
mpl.rcParams['ytick.labelsize'] = 23

In [ ]:
GC_name = 'M55'
path = os.getcwd()

In [ ]:
df = pd.read_csv("{}/{}.dat".format(path,GC_name),sep='\s+')
ZP = pd.read_csv("{}/{}_ZP_correct".format(path,GC_name),sep='\s+')

In [ ]:
plt.scatter(df['x'],df['y'],s=0.1);

In [ ]:
#determin center
x_guess,y_guess = 3000,3000
radius = 200
def centeroidnp(df):
    fit_stars = df[((df['x']-x_guess)**2 + (df['y']-y_guess)**2) < radius**2]
    length = len(fit_stars)
    x = fit_stars['x'].values
    y = fit_stars['y'].values
    sum_x = np.sum(x)
    sum_y = np.sum(y)
    return sum_x/length, sum_y/length
x_cen, y_cen = centeroidnp(df)
print("X Center is: {}".format(x_cen))
print("Y Center is: {}".format(y_cen))

In [ ]:
#determine MSTO and SGB
plt.scatter(df['VIvega'],df['Vvega'],s=0.1,alpha=0.5,c='grey')
plt.xlim(0.45,0.8)
plt.ylim(16,19)
plt.scatter(0.54,18.0,s=0.7,c='r',label='MSTO')
plt.scatter(0.54+0.05,17.28,s=0.7,c='b',label='SGB')
plt.legend()
plt.gca().invert_yaxis()

In [ ]:
M55_ridgeline = pd.read_csv("{}/fiducial_lines.csv".format(path))

In [ ]:
#plot the v vs i image within 3.5 mag from sgb
V_min = 17.28 - 3.5
V_max = 17.28 + 3.5
df_zeroth_cut = df[(df['Vvega'] < V_max) & (df['Vvega'] > V_min)].copy()
mask = (df_zeroth_cut['Vvega'] < 17.28 + 3.5) & (df_zeroth_cut['Vvega'] > 17.28 - 3.5)
df_first_cut = df_zeroth_cut[mask].copy()
df_first_cut['VIvega'] = df_first_cut['VIvega'].values - ZP['VI_diff'].values[mask]
vp = M55_ridgeline['v'].values
ip = M55_ridgeline['v'].values - M55_ridgeline['vi'].values
if vp[0] > vp[-1]:
    vp = np.flip(vp)
if ip[0] > ip[-1]:
    ip = np.flip(ip)
plt.scatter(df_first_cut['Vvega'],df_first_cut['Ivega'],s=0.1)
plt.xlabel('Vvega')
plt.ylabel('Ivega')
plt.plot(vp,ip,c='r')

In [ ]:
df_first_cut

In [ ]:
plt.scatter(df_first_cut['VIvega'],df_first_cut['Vvega'],s=0.1)
plt.xlabel('Vvega')
plt.ylabel('Ivega')
plt.plot(vp - ip,vp,c='r')
plt.xlim(0.4,1)
#plt.ylim(16,19)
plt.gca().invert_yaxis();

In [ ]:
Vvega = df_first_cut['Vvega'].values
Ivega = df_first_cut['Ivega'].values
VIvega = df_first_cut['VIvega'].values
d = []
mag_cut =0.08
ff_v = CubicSpline(ip,vp,extrapolate=True)
ff_i = CubicSpline(vp,ip,extrapolate=True)
Vvega_needed = ff_v(Ivega)
Ivega_needed = ff_i(Vvega)
mask = (((Vvega_needed + mag_cut) > Vvega) & (Vvega > (Vvega_needed - mag_cut)) & ((Ivega_needed + mag_cut) > Ivega) & (Ivega > (Ivega_needed - mag_cut)))
# for i in range(len(Ivega)):
#     Vvega_needed = ff_v(Ivega[i]) #new linear interpolation
#     Ivega_needed = ff_i(Vvega[i])
#     #use slightly bigger cut to avoid deleting too much points. A finer cut will be applied once the fiducial cmd is generated
#     if (((Vvega_needed + mag_cut) > Vvega[i]) & (Vvega[i] > (Vvega_needed - mag_cut)) & ((Ivega_needed + mag_cut) > Ivega[i]) & (Ivega[i] > (Ivega_needed - mag_cut))):
#         d.append(df_first_cut.iloc[i][['Vvega','err','Ivega','err.2','VIvega','err.1','x','y']].values.tolist())
# df_second_cut = pd.DataFrame(d,columns=['v','v_err','i','i_err','vi','vi_err','x','y'])
df_second_cut = df_first_cut[mask]
d = {'v':df_second_cut['Vvega'].values,'v_err':df_second_cut['err'].values,'i':df_second_cut['Ivega'].values,'i_err':df_second_cut['err.2'].values,'vi':df_second_cut['VIvega'].values,'vi_err':df_second_cut['err.1'].values,'x':df_second_cut['x'].values,'y':df_second_cut['y'].values}
df_second_cut = pd.DataFrame(d)

In [ ]:
plt.scatter(df_second_cut['vi'],df_second_cut['v'],s=0.1,alpha=0.8)
plt.plot(vp - ip,vp,c='r')
plt.xlim(0.4,1)
#plt.ylim(16,19)
plt.gca().invert_yaxis();

In [ ]:
df_second_cut

In [ ]:
df_second_cut.to_csv("{}/{}_fitstars_ZPCR_cmd.dat".format(path,GC_name),sep=',')